In [1]:
import pickle
from hepqpr.qallse import DataWrapper
from hepqpr.qallse.cli.func import process_response, process_response_vqe, process_response_eigensolver
from hepqpr.qallse.plotting import *

In [2]:
set_notebook_mode() # if running inside a notebook

# load the dataset and the response_neal (created using the qallse tool)
dw = DataWrapper.from_path('C:/Users/timsc/hepqpr-qallse-data/ds1/event000001000-hits.csv')
with open('C:/Users/timsc/hepqpr-qallse-data/ds1/evt1000-ds1-neal-0-response.pickle', 'rb') as f: 
    response_neal = pickle.load(f)

# process the response_neal and get the set of missing doublets
final_doublets, final_tracks = process_response(response_neal)
precision, recall, missings = dw.compute_score(final_doublets)

# plotting examples
iplot_results(dw, final_doublets, missings)
iplot_results(dw, final_doublets, missings, dims=list('zr'))
iplot_results_tracks(dw, final_tracks)

In [3]:
set_notebook_mode() # if running inside a notebook

# load the dataset and the response_vqe (created using the qallse tool)
dw = DataWrapper.from_path('C:/Users/timsc/hepqpr-qallse-data/ds1/event000001000-hits.csv')
with open('C:/Users/timsc/hepqpr-qallse-data/ds1/evt1000-ds1-vqe-0-response.pickle', 'rb') as f: 
    response_vqe = pickle.load(f)

# process the response_vqe and get the set of missing doublets
final_doublets, final_tracks, energy = process_response_vqe(response_vqe)
precision, recall, missings = dw.compute_score(final_doublets)

# plotting examples
iplot_results(dw, final_doublets, missings)
iplot_results(dw, final_doublets, missings, dims=list('zr'))
iplot_results_tracks(dw, final_tracks)

In [3]:
set_notebook_mode() # if running inside a notebook

# load the dataset and the response_eigensolver (created using the qallse tool)
dw = DataWrapper.from_path('C:/Users/timsc/hepqpr-qallse-data/ds1/event000001000-hits.csv')
with open('C:/Users/timsc/hepqpr-qallse-data/ds1/evt1000-ds1-eigensolver-0-response.pickle', 'rb') as f: 
    response_eigensolver = pickle.load(f)

# process the response_eigensolver and get the set of missing doublets
final_doublets, final_tracks, energy = process_response_eigensolver(response_eigensolver)
precision, recall, missings = dw.compute_score(final_doublets)

# plotting examples
iplot_results(dw, final_doublets, missings)
iplot_results(dw, final_doublets, missings, dims=list('zr'))
iplot_results_tracks(dw, final_tracks)

In [10]:
import sys
import os.path as op
sys.path.append('C:/Users/timsc/Documents/particle_tracking/code/')
from particle_tracking import *

In [11]:
sub_qubo_path = 'C:/Users/timsc/hepqpr-qallse-data/ds1/sub_qubos/'
sub_qubos = [op.join(sub_qubo_path, x) for x in os.listdir(sub_qubo_path)]
results_exact = []
for sub_qubo_file in sub_qubos:
    with open(sub_qubo_file, 'rb') as f: 
        sub_qubo = pickle.load(f)[1]
        b_ij, a_i, relations = prepare_data_dicts(sub_qubo)
        op = Tracking_Hamiltonian(b_ij, a_i)
        npme = NumPyMinimumEigensolver()
        result_exact = npme.compute_minimum_eigenvalue(operator=op)
        results_exact.append(result_exact)

In [21]:
precisions = []
recalls = []
for i in range(6):
    result_vqe_min_energy = min(response_vqe[i][1], key=lambda result_one_run: result_one_run['optimal_value'])
    precisions.append(precision(result_vqe_min_energy, results_exact[i]))
    recalls.append(recall(result_vqe_min_energy, results_exact[i]))

In [13]:
def precision(result_vqe, result_exact):

    precision = 0
    true_positives = 0
    dict_vqe=result_vqe['eigenstate']
    key_max=max(dict_vqe, key = lambda x: dict_vqe[x])
    positives = key_max.count('1')
    dict_exact=result_exact.eigenstate.to_dict_fn().sample()
    n_triplets = len(key_max)
    for i in range(n_triplets):
        if key_max[i]=='1' and list(dict_exact.keys())[0][i]=='1':
            true_positives += 1

    if positives!=0:
        precision = true_positives/positives

    return precision

def recall(result_vqe, result_exact):
    recall = 0
    true_positives = 0
    false_negatives = 0
    dict_vqe=result_vqe['eigenstate']
    key_max=max(dict_vqe, key = lambda x: dict_vqe[x])
    dict_exact=result_exact.eigenstate.to_dict_fn().sample()
    n_triplets = len(key_max)
    for i in range(n_triplets):
        if key_max[i]=='1' and list(dict_exact.keys())[0][i]=='1':
            true_positives += 1
        if key_max[i]=='0' and list(dict_exact.keys())[0][i]=='1':
            false_negatives += 1

    if true_positives+false_negatives!=0:
        recall = true_positives/(true_positives+false_negatives)

    return recall

In [22]:
precisions

[0.6, 0.0, 0.6, 0, 0.16666666666666666, 0.6]

In [23]:
recalls

[0.42857142857142855, 0.0, 0.375, 0.0, 0.14285714285714285, 0.375]

In [26]:
for x in results_exact:
    print(x.eigenstate.to_dict_fn().sample())

{'0100000110110001010': 1.0}
{'0010111010110010010': 1.0}
{'1001001011101001000': 1.0}
{'1100110101010000011': 1.0}
{'1100000100010101001': 1.0}
{'1010000011011001001': 1.0}


In [33]:
print(result_exact)

{   'aux_operator_eigenvalues': None,
    'eigenstate': VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False),
    'eigenvalue': (-62.43459336014448+0j)}
